# Librerias

In [1]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modo de conexion con el programa

- Abriremos automaticamente una ventana de Etabs

- Ingresaremos Informacion atraves de codigo Python

- Dibujaremos elementos frame y area atraves de codigo Python

In [2]:
# True: Trabajará con el programa ETBAS que esté abierto
# False: Se abrirá el programa de manera Automaticade 
AttachToInstance = True

# Complemento de conexion

In [3]:
# True: para especificar manualmente la ruta a ETABS.exe
# False: se usará la última versión instalada de ETABS
SpecifyPath = True

# si el indicador anterior está en True, especifique la ruta a ETABS a continuación
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 20\ETABS.exe"

# ruta completa del modelo 
# ajústelo a la ruta deseada de su modelo
APIPath = 'C:\CSi_ETABS_API_Example'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'API_1-001.edb'

# crear objeto API helper
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

if AttachToInstance:
    try:
        # obtener el objeto ETABS activo
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            # crear una instancia del objeto ETABS a partir de la ruta especificada
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            # crear una instancia del objeto ETABS a partir del último ETABS instalado
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    # Comenzar la aplicacion de ETABS
    myETABSObject.ApplicationStart()

# Crear un objeto SapModel
ret = SapModel = myETABSObject.SapModel

# Iniciar Modelo
ret = SapModel.InitializeNewModel()

0

# Definicion de espacio de trabajo

las unidades con las que inicia es kip, ft, F, se recomienda controlar las unidades de inicio
```python
# Crear un modelo en blanco
ret = SapModel.File.NewBlank()

# Crear Grillas
ret = SapModel.File.NewGridOnly(10,12,12,4,4,24,24)

# Crear plantilla de Steel Deck
ret = SapModel.File.NewSteelDeck(4,12,12,4,4,24,24)

# Abrir un archivo de Etabs
PathFile = "G:\Otros ordenadores\PRINCIPAL\ARCHIVOS 2023\EXPO PYTHON\Para Expo\Para Expo.edb"
ret = SapModel.File.OpenFile(PathFile)

# Guardar un archivo de Etabs
os.makedirs('C:\Ejemplo CEINT', exist_ok=True)
ret = SapModel.File.Save("C:\Ejemplo CEINT\example.edb")

```


In [ ]:
# Crear un modelo en blanco
ret =  SapModel.File.NewBlank()

# Definicion de materiales

- Concreto Armado   = 2
- Acero de Refuerzo = 6
- Albañileria       = 8

**Propiedades mecanicas del Concreto**

Crear dos tipos de concreto 280 y 210

Crear un material de Albañileria

In [20]:
# Cambiar unidades a kgf_m_C 
kgf_m_C = 8
ret = SapModel.SetPresentUnits(kgf_m_C)

# Definir Propiedades del Material
MATERIAL_MASONRY = 8
ret = SapModel.PropMaterial.SetMaterial('Albañileria CEINT',          # Name
                                   MATERIAL_MASONRY            # Material Type
                                        )

# Asignar propiedades mecanicas isotropicas al material en MPa
ret = SapModel.PropMaterial.SetMPIsotropic('Albañileria CEINT',            # Name
                                    1234,               # E
                                    0.1,                # v
                                    0.0000055           # T
                                    )

In [5]:
# Definir Propiedades del Material
MATERIAL_CONCRETE = 2
ret = SapModel.PropMaterial.SetMaterial('CONCRETO280',          # Name
                                   MATERIAL_CONCRETE            # Material Type
                                  )

In [6]:
# Asignar propiedades mecanicas isotropicas al material en MPa
ret = SapModel.PropMaterial.SetMPIsotropic('CONCRETO280',            # Name
                                    3600,               # E
                                    0.2,                # v
                                    0.0000055           # T
                                    )

In [7]:
# cambiar el peso especifico del material
ret = SapModel.PropMaterial.SetWeightAndMass('CONCRETO280', 1, 240)

**Propiedades Mecanicas del Acero de Refuerzo**

In [8]:
# Definir Propiedades del Material
MATERIAL_REBAR = 6
ret = SapModel.PropMaterial.SetMaterial('REBAR', 
                                   MATERIAL_REBAR
                                 )

# Asignar propiedades mecanicas isotropicas al material
ret = SapModel.PropMaterial.SetMPIsotropic('REBAR', 
                                    3600, 
                                    0.2, 
                                    0.0000055
                                    )

0

# Unidades de Trabajo

- kgf_m_C = 8
- Ton_m_C = 12
- Kg_cm_C = 14

In [10]:
# Cambiar unidades a kgf_m_C 
kgf_m_C = 8
ret = SapModel.SetPresentUnits(kgf_m_C)

In [12]:
ret = SapModel.View.RefreshView(0, False)

# Definir elementos 

## Frame

In [14]:
# Cambiar unidades a kgf_m_C 
kgf_m_C = 8
ret = SapModel.SetPresentUnits(kgf_m_C)

In [13]:
# FrameObj Interface            pag 2289
# Definir las propiedades de Seccion a un Frame rectangular   
ret = SapModel.PropFrame.SetRectangle('R1',       # Name del nuevo frame
                                'CONCRETO280',     # Material
                                12,         # b
                                12          # h
                                )

0

In [ ]:
# Datos extra para Frame Vigas        pag 3705
ret = SapModel.PropFrame.SetRebarBeam("R1",       # Name de un frame existente
                                "REBAR",    # Material del acero longitudinal
                                "REBAR",    # Material del acero transversal
                                3.5,        # Recubrimiento superior   6cm
                                3,          # Recubrimiento inferior   6cm
                                4.1,        # Top Left Area
                                4.2,        # Top Right Area
                                4.3,        # Bottom Left Area
                                4.4         # Bottom Right Area
                                )

In [ ]:
# Datos extra para Frame Columnas
ret = SapModel.PropFrame.SetRebarColumn("R1",     # Name de un frame existente
                                "REBAR",    # Material del acero longitudinal
                                "REBAR",    # Material del acero transversal
                                1,          # Pattern  (Rectangular = 1  /   Circular = 2)
                                1,          # Tipo de Cofinamiento  (Ties = 1  /   Spiral = 2)
                                2,          # Recubrimiento de la columna  4cm
                                10,         # Solo Aplica para la Columnas Circulares, Numero total de barras
                                4,          # Solo Aplica para la Columnas Rectangulares, Numero de barras en el eje local 3
                                3,          # Solo Aplica para la Columnas Rectangulares, Numero de barras en el eje local 2
                                "#10",      # El diametro del acero longitudinal
                                "#5",       # El diametro del acero transversal
                                4,          # Espaciamiento de los estribos
                                4,          # Numero de estribos alrededor del eje local 2
                                4,          # Numero de estribos alrededor del eje local 3
                                False)


In [ ]:
# Definir modificadores de propiedad a un Frame
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('R1', ModValue)

## Slab

Slab Type 
- Slab       = 0
- Drop       = 1
- Ribbed     = 3
- Waffle     = 4
- Mat        = 5
- Footing    = 6

Shell Type
- ShellThin  = 1
- Shellthick = 2
- Menbrane   = 3
- Layered    = 6

In [ ]:
# Definir las propiedades de la Losa Slab
ret = SapModel.PropArea.SetSlab("Losa Macisa CEINT",       # Name
                        0,                  # Slab Type - Slab
                        1,                  # ShellThin
                        "f'c=280kg/cm2",    # Material
                        15                  # Thickness
                        )

In [ ]:

# Inicializa la creacion de una Losa
ret = SapModel.PropArea.SetSlab("Slab Ribbed",    # Name
                            3,              # Ribbed
                            1,              # ShellThin
                            "CONC",         # Material
                            10              # Thickness
                        )
# Definir las propiedades de la Losa Ribbed
ret = SapModel.PropArea.SetSlabRibbed(
                            "Slab Ribbed",  # Name
                            11.1,           # OverallDepth              
                            22.2,           # SlabThickness
                            33.3,           # StemWidthTop
                            44.4,           # StemWidthBottom
                            55.5,           # RibSpacing
                            2               # RibsParallelTo - Local Axis
                            )

## Wall

WallPropType
- Specified      = 1
- AutoSelectList = 2

Shell Type
- ShellThin  = 1
- Shellthick = 2
- Menbrane   = 3
- Layered    = 6

In [ ]:
ret = SapModel.PropArea.SetWall("Placa",      # Name
                        1,              # Material
                        1,              # Shell Type - Shell Thin   
                        "CONC",         # Material Type
                        20              # Thickness
                        )

Modificador de propiedades

* Value(0) Membrane f11 modifier
* Value(1) Membrane f22 modifier
* Value(2) Membrane f12 modifier
* Value(3) Bending m11 modifier
* Value(4) Bending m22 modifier
* Value(5) Bending m12 modifier
* Value(6) Shear v13 modifier
* Value(7) Shear v23 modifier
* Value(8) Mass modifier
* Value(9) Weight modifier

In [ ]:
# MyValue = [1, 1, 1, 1, 1, 1, 1, 1, 1]
# ret = SapModel.PropArea.SetModifiers("Slab", MyValue)

# Dibujar elemento Frame

In [ ]:
# Añadir objeto Frame por Coordenadas
FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 10, FrameName1, 'R1', '1', 'Global')
[FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1', '2', 'Global')
[FrameName3, ret] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1', '3', 'Global')

# Crear LoadPattern

Definicion de Load Patterns

- Dead          = 1
- SuperDead     = 2
- Live          = 3
- ReduceLive    = 4
- Quake         = 5
- Wind          = 6
- Snow          = 7
- Other         = 8

In [ ]:

ret = SapModel.LoadPatterns.Add('1',    # Name del Load Pattern
                                1,      # Tipo de Load Pattern
                                1,      # Considerar peso propio - Factor
                                True    # Opcional - True: Crear un nuevo caso de carga
                                )


In [ ]:
# Ejemplo - añadiendo LoadPatterns
LTYPE_OTHER = 8
ret = SapModel.LoadPatterns.Add('1', LTYPE_OTHER, 1, True)
ret = SapModel.LoadPatterns.Add('2', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('3', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('4', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('5', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('6', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('7', LTYPE_OTHER, 0, True)

# Asignar Carga

In [ ]:
#assign loading for load pattern 2
PointName1 = ' '
PointName2 = ' '
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0,0,-10,0,0,0]
ret = SapModel.PointObj.SetLoadForce(PointName1, '2', PointLoadValue)
ret = SapModel.FrameObj.SetLoadDistributed(FrameName3, '2', 1, 10, 0, 1, 1.8, 1.8)


#assign loading for load pattern 3
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0,0,-17.2,0,-54.4,0]
ret = SapModel.PointObj.SetLoadForce(PointName2, '3', PointLoadValue)

#assign loading for load pattern 4
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '4', 1, 11, 0, 1, 2, 2)

#assign loading for load pattern 5
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '5', 1, 2, 0, 1, 2, 2, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '5', 1, 2, 0, 1, -2, -2, 'Local')

#assign loading for load pattern 6
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '6', 1, 2, 0, 1, 0.9984, 0.3744, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '6', 1, 2, 0, 1, -0.3744, 0, 'Local')

#assign loading for load pattern 7
ret = SapModel.FrameObj.SetLoadPoint(FrameName2, '7', 1, 2, 0.5, -15, 'Local')


# Asignar Restricciones

In [ ]:
# Asignar resitricciones al punto de la Base
PointName1 = ' '
PointName2 = ' '
Restraint = [True, True, True, True, False, False]

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

# Asignar restricciones al punto superior
Restraint = [True, True, False, False, False, False]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName2, Restraint)

# refinaciones

Asignar Brazo rigido       pag 2464
```python
'assign offsets
SapModel.FrameObj.SetEndLengthOffset("15", False, 12, 12, 0.5)
```
---

Asignar Releases pag 2515
```python
'assign end releases
ReDim ii(5)
ReDim jj(5)
ReDim StartValue(5)
ReDim EndValue(5)
ii(5) = True
jj(5) = True
ret =SapModel.FrameObj.SetReleases("13", ii, jj, StartValue, EndValue)
```
---

Asignar propiedad de una seccion a un frame seleccionado       pag 2521
```python
'create new concrete frame section property
 ret = SapModel.PropFrame.SetRectangle("R1", "4000Psi", 20, 12)
'set frame section property
 ret = SapModel.FrameObj.SetSection("8", "R1")
```
---